In [1]:
#wuhuu 1.93 kaldi 0.30 gelistirme

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np



In [319]:
def aggregate_weather_data(daily_weather):
        
        daily_weather['tarih'] = pd.to_datetime(daily_weather['tarih'])
        daily_weather['day'] = daily_weather['tarih'].dt.date
        daily_weather['hour'] = daily_weather['tarih'].dt.hour


        daily_weather = daily_weather.groupby(['ilce', pd.Grouper(freq='D', key='tarih')])
        ## Apply the aggregations
        daily_weather= daily_weather.agg({
            't_2m:C': ['max', 'min', 'mean','std'],  # temperature
            'prob_precip_1h:p': ['sum', 'max' ,'mean'],  # precipitation
            'wind_speed_10m:ms': ['max', 'mean','std'],  # wind speed
            'wind_dir_10m:d': 'mean',  # wind direction
            'global_rad:W': 'sum',  # sunshine duration
            'effective_cloud_cover:p': ['mean','std'],  # cloud cover
            'relative_humidity_2m:p': ['max', 'min', 'mean']  # humidity
        })

        # Flatten the MultiIndex columns
        daily_weather.columns = ['_'.join(col).strip() for col in daily_weather.columns.values]
        daily_weather= daily_weather.reset_index() 
        return daily_weather
def create_unique_id(df):
    df['unique_id'] = df['tarih'].astype(str) +  '-' +df['ilce'].astype(str) 
    return df

In [320]:
from datetime import datetime
train= pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")
weather = pd.read_csv("weather.csv")
holidays = pd.read_csv("holidays.csv")

# "date" sütununu "tarih" olarak değiştirme
weather.rename(columns={"date": "tarih"}, inplace=True)
weather.rename(columns={"name": "ilce"}, inplace=True)


citylatlong = weather[['ilce', 'lat', 'lon']]
citylatlong

,ilce,lat,lon
0,Manisa-Ahmetli,38.6184,28.6712
1,Manisa-Ahmetli,38.6184,28.6712
2,Manisa-Ahmetli,38.6184,28.6712
3,Manisa-Ahmetli,38.6184,28.6712
4,Manisa-Ahmetli,38.6184,28.6712
...,...,...,...
1302835,Izmir-Odemis,38.2283,27.9748
1302836,Izmir-Odemis,38.2283,27.9748
1302837,Izmir-Odemis,38.2283,27.9748
1302838,Izmir-Odemis,38.2283,27.9748


In [321]:

# Tüm ilçe isimlerini küçük harfe dönüştür
weather["ilce"] = weather["ilce"].str.lower()
train["ilce"] = train["ilce"].str.lower()
test["ilce"] = test["ilce"].str.lower()

print(len(test))
print(len(train))





1363
48148


In [322]:
holidays = pd.read_csv('holidays.csv')
holidays.columns =['year', 'month' ,'day' , 'holiday']
add_columns = holidays['holiday']
# Create a new column for each unique holiday name

holidays['tarih'] = pd.to_datetime(holidays[['year', 'month', 'day']])
for new in holidays['holiday'].unique():
    holidays[new] = (holidays['holiday'] == new).astype(int)
holidays.drop(['holiday', 'year', 'month', 'day'] , axis=1 , inplace=True)
holidays.columns
holidays.tail()


,tarih,New Year's Day,National Sovereignty and Children's Day,Labour Day,"Commemoration of Ataturk, Youth and Sports Day",Democracy and National Unity Day,Victory Day,Republic Day,Ramadan Feast* (*estimated),Ramadan Feast Holiday* (*estimated),Sacrifice Feast* (*estimated),Sacrifice Feast Holiday* (*estimated),National Sovereignty and Children's Day; Ramadan Feast Holiday* (*estimated)
50,2024-04-12,0,0,0,0,0,0,0,0,1,0,0,0
51,2024-06-16,0,0,0,0,0,0,0,0,0,1,0,0
52,2024-06-17,0,0,0,0,0,0,0,0,0,0,1,0
53,2024-06-18,0,0,0,0,0,0,0,0,0,0,1,0
54,2024-06-19,0,0,0,0,0,0,0,0,0,0,1,0


In [323]:
weather.rename(columns={'date': 'tarih','name':'ilce'}, inplace=True)
weather = aggregate_weather_data(weather)
#weather = create_unique_id(weather)
#weather['unique_id']


In [324]:

after_date = datetime(2024, 2, 1)
# train data buraya baslangictan 2024/2/01'a kadar olacak
weather_train = weather[weather['tarih'] < after_date]
holidays_train = holidays[holidays['tarih'] < after_date]

# test data buraya sadece asagidaki tarihten itibaren olacak
weather_test = weather[weather['tarih'] >= after_date ]
holidays_test = holidays[holidays['tarih'] >= after_date]
len(weather_test)


1363

In [325]:
#ARADA BOSLUK GUNLER VAR ONLARI MEAN ILE DOLDURDUK SORUN OLABILIR TABI AMA OLSUN
from sklearn.impute import SimpleImputer


columns_to_impute = ['bildirimsiz_sum', 'bildirimli_sum']

# Create a simple imputer
imputer = SimpleImputer(strategy='mean')

# Fit the imputer to the data
imputer.fit(train[columns_to_impute])

# Impute the missing values
train[columns_to_impute] = imputer.transform(train[columns_to_impute])
train.tail()

,tarih,ilce,bildirimsiz_sum,bildirimli_sum
48143,2024-01-23,manisa-ahmetli,1.0,0.0
48144,2024-01-24,manisa-ahmetli,2.0,0.0
48145,2024-01-25,manisa-ahmetli,2.0,0.0
48146,2024-01-26,manisa-ahmetli,2.0,0.0
48147,2024-01-29,manisa-ahmetli,0.0,1.0


In [326]:
train.columns = ['tarih' , 'ilce' , 'bildirimsiz_sum', 'bildirimli_sum']
#train = create_unique_id(train)
train['tarih'] = pd.to_datetime(train['tarih'])
 #Process data and train model
train

,tarih,ilce,bildirimsiz_sum,bildirimli_sum
0,2021-01-01,izmir-aliaga,5.0,0.0
1,2021-01-02,izmir-aliaga,13.0,0.0
2,2021-01-03,izmir-aliaga,4.0,0.0
3,2021-01-04,izmir-aliaga,9.0,0.0
4,2021-01-05,izmir-aliaga,2.0,0.0
...,...,...,...,...
48143,2024-01-23,manisa-ahmetli,1.0,0.0
48144,2024-01-24,manisa-ahmetli,2.0,0.0
48145,2024-01-25,manisa-ahmetli,2.0,0.0
48146,2024-01-26,manisa-ahmetli,2.0,0.0


In [327]:
test.columns= ['tarih','ilce','bildirimli_sum']
#test = create_unique_id(test)
test.head()
print(len(test))

1363


In [328]:

sample_submission.tail()

,unique_id,bildirimsiz_sum
1358,2024-02-29-izmir-beydag,NaN
1359,2024-02-29-izmir-narlidere,NaN
1360,2024-02-29-izmir-selcuk,NaN
1361,2024-02-29-manisa-kula,NaN
1362,2024-02-29-manisa-ahmetli,NaN


In [329]:
train.sample()

,tarih,ilce,bildirimsiz_sum,bildirimli_sum
3791,2022-05-12,izmir-bergama,0.0,1.0


In [330]:
##@@@ test ops train ops
test['tarih'] = pd.to_datetime(test['tarih'])

test['days_in_month'] = test['tarih'].dt.days_in_month
test['days_in_year'] = test['tarih'].dt.day_of_year
test['days_in_week'] = test['tarih'].dt.weekday
test['month'] = test['tarih'].dt.month
test['year'] = test['tarih'].dt.year

train['tarih'] = pd.to_datetime(train['tarih'])

train['days_in_month'] = train['tarih'].dt.days_in_month
train['days_in_year'] = train['tarih'].dt.day_of_year
train['days_in_week'] = train['tarih'].dt.weekday
train['month'] = train['tarih'].dt.month
train['year'] = train['tarih'].dt.year


# Tarih sütunlarının formatlarını uygun hale getir
weather["tarih"] = pd.to_datetime(weather["tarih"])
train["tarih"] = pd.to_datetime(train["tarih"])
test["tarih"] = pd.to_datetime(test["tarih"])
max(test['tarih'])
len(test)

1363

In [331]:
from sklearn.calibration import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train['ilce'])
# Transform both datasets using the same encoder
train['ilce-value'] = label_encoder.transform(train['ilce'])
test['ilce-value'] = label_encoder.transform(test['ilce'])


In [332]:
# Set ilce and tarih as the index
test.set_index(['ilce', 'tarih'], inplace=True)
weather_test.set_index(['ilce', 'tarih'], inplace=True)
train.set_index(['ilce', 'tarih'], inplace=True)
weather_train.set_index(['ilce', 'tarih'], inplace=True)





In [333]:

# Merge the dataframes
test_df = pd.merge(test, weather_test, on=['ilce', 'tarih'])
train_df = pd.merge(train, weather_train, on=['ilce', 'tarih'])
train_df = pd.merge(train, weather_train, left_index=True, right_index=True)
len(test_df)
test_df.columns

Index(['bildirimli_sum', 'days_in_month', 'days_in_year', 'days_in_week',
       'month', 'year', 'ilce-value', 't_2m:C_max', 't_2m:C_min',
       't_2m:C_mean', 't_2m:C_std', 'prob_precip_1h:p_sum',
       'prob_precip_1h:p_max', 'prob_precip_1h:p_mean',
       'wind_speed_10m:ms_max', 'wind_speed_10m:ms_mean',
       'wind_speed_10m:ms_std', 'wind_dir_10m:d_mean', 'global_rad:W_sum',
       'effective_cloud_cover:p_mean', 'effective_cloud_cover:p_std',
       'relative_humidity_2m:p_max', 'relative_humidity_2m:p_min',
       'relative_humidity_2m:p_mean'],
      dtype='object')

Creating Weather data and merging them into train data 

In [334]:
train_index_values = train_df.index.get_level_values('tarih')
train_df['tarih-value'] = pd.factorize(train_index_values)[0]
max_train_tarih = max(train_df['tarih-value'])

test_index_values = test_df.index.get_level_values('tarih')
test_df['tarih-value'] = pd.factorize(test_index_values)[0] + max_train_tarih + 1

test_df

,,bildirimli_sum,days_in_month,days_in_year,days_in_week,month,year,ilce-value,t_2m:C_max,t_2m:C_min,t_2m:C_mean,...,wind_speed_10m:ms_mean,wind_speed_10m:ms_std,wind_dir_10m:d_mean,global_rad:W_sum,effective_cloud_cover:p_mean,effective_cloud_cover:p_std,relative_humidity_2m:p_max,relative_humidity_2m:p_min,relative_humidity_2m:p_mean,tarih-value
ilce,tarih,,,,,,,,,,,,,,,,,,,,,
izmir-aliaga,2024-02-01,0,29,32,3,2,2024,0,12.0,5.0,7.616667,...,4.941667,1.155297,34.337500,3423.1,10.037500,11.238707,85.3,49.9,72.116667,1126
izmir-bayindir,2024-02-01,1,29,32,3,2,2024,2,12.6,2.9,6.429167,...,2.212500,0.401424,95.187500,3413.1,13.562500,22.820677,94.1,52.2,77.904167,1126
izmir-bayrakli,2024-02-01,0,29,32,3,2,2024,3,12.0,3.9,6.954167,...,2.395833,0.603237,36.870833,3476.9,9.041667,12.470798,92.0,49.8,76.537500,1126
izmir-bergama,2024-02-01,1,29,32,3,2,2024,4,11.4,2.8,6.575000,...,3.670833,1.242186,51.166667,3195.0,34.620833,33.273177,87.4,51.5,73.729167,1126
izmir-bornova,2024-02-01,1,29,32,3,2,2024,6,11.3,2.9,6.054167,...,2.595833,0.670807,31.016667,3460.2,12.441667,17.586480,97.2,50.1,79.058333,1126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
izmir-beydag,2024-02-29,0,29,60,3,2,2024,5,22.6,8.7,15.387500,...,1.141667,0.281172,150.458333,4712.2,16.316667,26.080555,93.7,32.8,64.316667,1154
izmir-narlidere,2024-02-29,0,29,60,3,2,2024,23,21.2,12.6,16.283333,...,2.012500,0.619476,156.383333,4523.8,25.470833,34.710473,84.5,45.3,68.395833,1154
izmir-selcuk,2024-02-29,0,29,60,3,2,2024,26,23.2,8.8,15.429167,...,1.670833,0.603957,126.637500,4548.3,19.912500,29.647880,90.3,39.0,69.895833,1154


In [335]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# Hedef değişkeni ve özellikleri ayır
X = train_df.drop(columns=["bildirimsiz_sum"])
y = train_df["bildirimsiz_sum"]

corr = train_df.corr()
target_corr = abs(corr["bildirimsiz_sum"])
corr_threshold = 0.02
high_corr_features = target_corr[target_corr > corr_threshold]
# özellik isimlerini alalım ve bildirimsiz_sum özelliğini çıkaralım
hcf_names = [k for k, v in high_corr_features.items()]; hcf_names.remove("bildirimsiz_sum")
#print(hcf_names)
features= ['bildirimli_sum', 'days_in_month', 'days_in_year',
       'days_in_week', 'month', 'year', 't_2m:C_max', 't_2m:C_min',
       't_2m:C_mean', 't_2m:C_std', 'prob_precip_1h:p_sum',
       'prob_precip_1h:p_max', 'prob_precip_1h:p_mean',
       'wind_speed_10m:ms_max', 'wind_speed_10m:ms_mean',
       'wind_speed_10m:ms_std', 'wind_dir_10m:d_mean', 'global_rad:W_sum',
       'effective_cloud_cover:p_mean', 'effective_cloud_cover:p_std',
       'relative_humidity_2m:p_max', 'relative_humidity_2m:p_min',
       'relative_humidity_2m:p_mean', 'tarih-value', 'ilce-value']

In [336]:
test_df.columns




Index(['bildirimli_sum', 'days_in_month', 'days_in_year', 'days_in_week',
       'month', 'year', 'ilce-value', 't_2m:C_max', 't_2m:C_min',
       't_2m:C_mean', 't_2m:C_std', 'prob_precip_1h:p_sum',
       'prob_precip_1h:p_max', 'prob_precip_1h:p_mean',
       'wind_speed_10m:ms_max', 'wind_speed_10m:ms_mean',
       'wind_speed_10m:ms_std', 'wind_dir_10m:d_mean', 'global_rad:W_sum',
       'effective_cloud_cover:p_mean', 'effective_cloud_cover:p_std',
       'relative_humidity_2m:p_max', 'relative_humidity_2m:p_min',
       'relative_humidity_2m:p_mean', 'tarih-value'],
      dtype='object')

In [337]:
X_test = test_df[features]
X_test

,,bildirimli_sum,days_in_month,days_in_year,days_in_week,month,year,t_2m:C_max,t_2m:C_min,t_2m:C_mean,t_2m:C_std,...,wind_speed_10m:ms_std,wind_dir_10m:d_mean,global_rad:W_sum,effective_cloud_cover:p_mean,effective_cloud_cover:p_std,relative_humidity_2m:p_max,relative_humidity_2m:p_min,relative_humidity_2m:p_mean,tarih-value,ilce-value
ilce,tarih,,,,,,,,,,,,,,,,,,,,,
izmir-aliaga,2024-02-01,0,29,32,3,2,2024,12.0,5.0,7.616667,2.474156,...,1.155297,34.337500,3423.1,10.037500,11.238707,85.3,49.9,72.116667,1126,0
izmir-bayindir,2024-02-01,1,29,32,3,2,2024,12.6,2.9,6.429167,3.455491,...,0.401424,95.187500,3413.1,13.562500,22.820677,94.1,52.2,77.904167,1126,2
izmir-bayrakli,2024-02-01,0,29,32,3,2,2024,12.0,3.9,6.954167,2.861738,...,0.603237,36.870833,3476.9,9.041667,12.470798,92.0,49.8,76.537500,1126,3
izmir-bergama,2024-02-01,1,29,32,3,2,2024,11.4,2.8,6.575000,2.612387,...,1.242186,51.166667,3195.0,34.620833,33.273177,87.4,51.5,73.729167,1126,4
izmir-bornova,2024-02-01,1,29,32,3,2,2024,11.3,2.9,6.054167,2.982263,...,0.670807,31.016667,3460.2,12.441667,17.586480,97.2,50.1,79.058333,1126,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
izmir-beydag,2024-02-29,0,29,60,3,2,2024,22.6,8.7,15.387500,4.698040,...,0.281172,150.458333,4712.2,16.316667,26.080555,93.7,32.8,64.316667,1154,5
izmir-narlidere,2024-02-29,0,29,60,3,2,2024,21.2,12.6,16.283333,3.040547,...,0.619476,156.383333,4523.8,25.470833,34.710473,84.5,45.3,68.395833,1154,23
izmir-selcuk,2024-02-29,0,29,60,3,2,2024,23.2,8.8,15.429167,5.090783,...,0.603957,126.637500,4548.3,19.912500,29.647880,90.3,39.0,69.895833,1154,26


In [338]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 10],
    'n_estimators': [50, 100, 200],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 1.0],
    'gamma': [0, 0.1, 0.3]
}

# Model oluşturma
xgb = XGBRegressor()

# GridSearchCV ile en iyi parametre kombinasyonunun bulunması
#grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
#grid_search.fit(X,y)
#njobs -1 olmali (benim makinem paralel calismiyor cunku )
# En iyi parametrelerin bulunması
#best_params = grid_search.best_params_
best_params = {'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 50, 'subsample': 0.8}
print("En iyi parametreler:", best_params)


En iyi parametreler: {'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 50, 'subsample': 0.8}


In [339]:

# XGBoost modelini eğitme
xgboost_model = XGBRegressor(**best_params)
xgboost_model.fit(train_df[features],y)

# CatBoost modelini eğitme
catboost_model = CatBoostRegressor()
catboost_model.fit(train_df[features],y)

# Tahminler yapma



X_test = test_df[features]
X_test


Learning rate set to 0.075514
0:	learn: 5.3873193	total: 5.21ms	remaining: 5.2s
1:	learn: 5.3534278	total: 9.92ms	remaining: 4.95s
2:	learn: 5.3226892	total: 14.5ms	remaining: 4.81s
3:	learn: 5.3007801	total: 19ms	remaining: 4.73s
4:	learn: 5.2798363	total: 23.6ms	remaining: 4.7s
5:	learn: 5.2576738	total: 28ms	remaining: 4.63s
6:	learn: 5.2292858	total: 32.5ms	remaining: 4.62s
7:	learn: 5.2145966	total: 37ms	remaining: 4.59s
8:	learn: 5.1985771	total: 41.5ms	remaining: 4.57s
9:	learn: 5.1736638	total: 46.3ms	remaining: 4.58s
10:	learn: 5.1464641	total: 50.5ms	remaining: 4.54s
11:	learn: 5.1263318	total: 54.8ms	remaining: 4.51s
12:	learn: 5.1093030	total: 58.6ms	remaining: 4.45s
13:	learn: 5.0944631	total: 62.1ms	remaining: 4.38s
14:	learn: 5.0780828	total: 65.8ms	remaining: 4.32s
15:	learn: 5.0608869	total: 69.4ms	remaining: 4.27s
16:	learn: 5.0483243	total: 73.8ms	remaining: 4.27s
17:	learn: 5.0384244	total: 77.6ms	remaining: 4.23s
18:	learn: 5.0308986	total: 81.1ms	remaining: 4.19s


,,bildirimli_sum,days_in_month,days_in_year,days_in_week,month,year,t_2m:C_max,t_2m:C_min,t_2m:C_mean,t_2m:C_std,...,wind_speed_10m:ms_std,wind_dir_10m:d_mean,global_rad:W_sum,effective_cloud_cover:p_mean,effective_cloud_cover:p_std,relative_humidity_2m:p_max,relative_humidity_2m:p_min,relative_humidity_2m:p_mean,tarih-value,ilce-value
ilce,tarih,,,,,,,,,,,,,,,,,,,,,
izmir-aliaga,2024-02-01,0,29,32,3,2,2024,12.0,5.0,7.616667,2.474156,...,1.155297,34.337500,3423.1,10.037500,11.238707,85.3,49.9,72.116667,1126,0
izmir-bayindir,2024-02-01,1,29,32,3,2,2024,12.6,2.9,6.429167,3.455491,...,0.401424,95.187500,3413.1,13.562500,22.820677,94.1,52.2,77.904167,1126,2
izmir-bayrakli,2024-02-01,0,29,32,3,2,2024,12.0,3.9,6.954167,2.861738,...,0.603237,36.870833,3476.9,9.041667,12.470798,92.0,49.8,76.537500,1126,3
izmir-bergama,2024-02-01,1,29,32,3,2,2024,11.4,2.8,6.575000,2.612387,...,1.242186,51.166667,3195.0,34.620833,33.273177,87.4,51.5,73.729167,1126,4
izmir-bornova,2024-02-01,1,29,32,3,2,2024,11.3,2.9,6.054167,2.982263,...,0.670807,31.016667,3460.2,12.441667,17.586480,97.2,50.1,79.058333,1126,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
izmir-beydag,2024-02-29,0,29,60,3,2,2024,22.6,8.7,15.387500,4.698040,...,0.281172,150.458333,4712.2,16.316667,26.080555,93.7,32.8,64.316667,1154,5
izmir-narlidere,2024-02-29,0,29,60,3,2,2024,21.2,12.6,16.283333,3.040547,...,0.619476,156.383333,4523.8,25.470833,34.710473,84.5,45.3,68.395833,1154,23
izmir-selcuk,2024-02-29,0,29,60,3,2,2024,23.2,8.8,15.429167,5.090783,...,0.603957,126.637500,4548.3,19.912500,29.647880,90.3,39.0,69.895833,1154,26


In [340]:
print(len(test_df))

1363


In [341]:


#catboost_preds = catboost_model.predict(X_test)
xgboost_preds = xgboost_model.predict(X_test)
catboost_preds = catboost_model.predict(X_test)
print("----------------------------------")
print(len(X_test))
print(catboost_preds)


----------------------------------
1363
[5.36427269 3.11366253 4.33173584 ... 1.58147091 2.9867291  1.5142571 ]


In [342]:

ensemble_preds = (catboost_preds + xgboost_preds) / 2
ensemble_preds=np.round(catboost_preds).astype(np.int8)
# Sample submission dosyasına tahminleri ekleyerek yeni bir dosya oluşturma
submission = sample_submission.copy()
print(ensemble_preds)
submission["bildirimsiz_sum"] = ensemble_preds
submission.to_csv("ensemble3_submission-emre-8-05-24.csv", index=False)
#model2.py

[5 3 4 ... 2 3 2]
